In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [5]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [6]:
dataset["classification_yes"].value_counts()

1    249
0    150
Name: classification_yes, dtype: int64

In [7]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes'  ]]
dep=dataset[['classification_yes' ]]

In [8]:
indep.shape

(399, 27)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x_train=sc.fit_transform(X_train)
x_test=sc.transform(X_test)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
y_train = y_train.values.ravel()
param_grid={'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],'tol':[ 1e-3],'max_iter':[2000]},
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid=({'max_iter': [2000],
                          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
                          'tol': [0.001]},),
             scoring='f1_weighted', verbose=3)

In [12]:
results = pd.DataFrame(grid.cv_results_)
print(results[['params', 'mean_test_score', 'std_test_score']])

                                              params  mean_test_score  \
0  {'max_iter': 2000, 'solver': 'newton-cg', 'tol...         0.988760   
1  {'max_iter': 2000, 'solver': 'lbfgs', 'tol': 0...         0.988760   
2  {'max_iter': 2000, 'solver': 'liblinear', 'tol...         0.973905   
3  {'max_iter': 2000, 'solver': 'saga', 'tol': 0....         0.985047   

   std_test_score  
0        0.009179  
1        0.009179  
2        0.008965  
3        0.007478  


In [13]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_iter,param_solver,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.031423,0.024097,0.010977,0.006232,2000,newton-cg,0.001,"{'max_iter': 2000, 'solver': 'newton-cg', 'tol...",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
1,0.022170,0.011834,0.001600,0.003200,2000,lbfgs,0.001,"{'max_iter': 2000, 'solver': 'lbfgs', 'tol': 0...",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
2,0.035160,0.056020,0.003126,0.006252,2000,liblinear,0.001,"{'max_iter': 2000, 'solver': 'liblinear', 'tol...",0.963284,0.981233,0.962573,0.981217,0.981217,0.973905,0.008965,4
3,0.051152,0.029629,0.003379,0.004148,2000,saga,0.001,"{'max_iter': 2000, 'solver': 'saga', 'tol': 0....",0.981569,0.981233,0.981217,1.000000,0.981217,0.985047,0.007478,3


In [14]:
bestParams=grid.best_params_
bestEstimator=grid.best_estimator_
bestScore=grid.best_score_
print("Best parameter {} ".format(bestParams))
print("Best Estimator {} ".format(bestEstimator))
print("Best Score {} ".format(bestScore))

Best parameter {'max_iter': 2000, 'solver': 'newton-cg', 'tol': 0.001} 
Best Estimator LogisticRegression(max_iter=2000, solver='newton-cg', tol=0.001) 
Best Score 0.9887599962028955 


In [15]:
gridPred=grid.predict(x_test)

In [16]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,gridPred)
print(cm)

[[51  0]
 [ 1 81]]


In [17]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,gridPred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

1.0

In [19]:
from sklearn.metrics import f1_score
fscore=f1_score(y_test,gridPred,average='weighted')
print(fscore)

0.9924946382275899


In [20]:
indep.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes'],
      dtype='object')

In [22]:
age=int(input("AGE:"))
bp=float(input("BP:"))
al=float(input("AL:"))
su=int(input("SU:"))
bgr=float(input("BGR:"))
bu=float(input("BU:"))
sc=int(input("SC:"))
sod=float(input("SOD"))
pot=float(input("POT:"))
hrmo=float(input("HRMO:"))
pcv=int(input("PCV:"))
wc=int(input("WC:"))
rc=float(input("RC:"))
sg_b=int(input("SG B:"))
sg_c=int(input("SG C:"))
sg_d=int(input("SG D:"))
sg_e=int(input("SG E:"))
rbc_normal=int(input("RBC NORMAL:"))
pc_normal=int(input("PC NORMAL:"))
pcc_present=int(input("PC PRESENT:"))
ba_present=int(input("BA PRESENT:"))
htn_yes=int(input("HTN YES:"))
dm_yes=int(input("DM YES:"))
cad_yes=int(input("CAD YES:"))
appet_yes=int(input("APPET YES:"))
pe_yes=int(input("PE YES"))
ane_yes=int(input("ANE YES:"))

AGE:21
BP:55.34
AL:2.3
SU:2
BGR:89.3
BU:34.3
SC:2
SOD120.4
POT:2.3
HRMO:7.4
PCV:23
WC:1200
RC:2.5
SG B:0
SG C:1
SG D:0
SG E:0
RBC NORMAL:1
PC NORMAL:0
PC PRESENT:1
BA PRESENT:1
HTN YES:1
DM YES:0
CAD YES:0
APPET YES:1
PE YES1
ANE YES:0


In [23]:
Prediction=grid.predict([[age,bp,al,su, bgr, bu, sc, sod, pot, hrmo, pcv,
       wc, rc, sg_b, sg_c, sg_d, sg_e, rbc_normal, pc_normal,
       pcc_present, ba_present, htn_yes, dm_yes, cad_yes,
       appet_yes, pe_yes, ane_yes]])

In [24]:
print(" Prediction Value : {} ".format(Prediction))

 Prediction Value : [1] 


In [25]:
import pickle
fileName="FinalModel_LR.sav"
pickle.dump(grid,open(fileName,'wb'))

In [26]:
loadedModel=pickle.load(open("FinalModel_LR.sav",'rb'))
result=loadedModel.predict([[age,bp,al,su, bgr, bu, sc, sod, pot, hrmo, pcv,
       wc, rc, sg_b, sg_c, sg_d, sg_e, rbc_normal, pc_normal,
       pcc_present, ba_present, htn_yes, dm_yes, cad_yes,
       appet_yes, pe_yes, ane_yes]])
print("Future Predictions : {} ".format(result))

Future Predictions : [1] 
